In [1]:
import os


os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["WANDB_PROJECT"] = "gadgets"
os.environ["WANDB_ENTITY"] = "transformersclub"

In [2]:
import transformers
import trl
import pandas as pd
import datasets
import torch
import wandb

wandb.init(group="dpo")

/var/tmp/xkadlci2/.conda/envs/calc-x/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/var/tmp/xkadlci2/.conda/envs/calc-x/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
[2024-01-25 22:24:00,634] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: prompteus (transformersclub). Use `wandb login --relogin` to force relogin


In [3]:
import gadgets

model_id = "MU-NLPC/calcformer-t5-large"

model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, use_fast=False)
model_class = gadgets.model.gadget_assisted_model(model.__class__)
model = model_class.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model.prepare_for_generate(
    tokenizer, 
    enabled_gadgets=[gadgets.gadget.Calculator()], 
    default_max_tokens=512
)

In [5]:
prompt = """
The profit from a business transaction is shared among 2 business partners, 
Mike and Johnson in the ratio 2:5 respectively. 
If Johnson got $2500, how much will Mike have 
after spending some of his share on a shirt that costs $200?
""".strip()

inputs = tokenizer(prompt, return_tensors="pt")
output_ids = model.generate(**inputs)
tokenizer.decode(output_ids[0], spaces_between_special_tokens=False)

'According to the ratio, for every 5 parts that Johnson gets, Mike gets 2 parts Since Johnson got $2500, each part is therefore $2500/5 = $<gadget id="calculator">2500/5</gadget><output>500</output> 500 Mike will get 2*$500 = $<gadget id="calculator">2*500</gadget><output>1_000</output> 1000 After buying the shirt he will have $1000-$200 = $<gadget id="calculator">1000-200</gadget><output>800</output> 800 left. Final result is<result>800</result></s>'

In [6]:
dummy_dataset = [
    {
        "prompt": "Xiao Wang wants to dilute 150 kg of pesticides with a 20% drug content into a 5% drug solution. How many kilograms of water need to be added?",
        "rejected": '<gadget id="calculator">20 / 100</gadget> <output>1/5 = around 0.2</output> <gadget id="calculator">5 / 100</gadget> <output>1/20 = around 0.05</output> <gadget id="calculator">(1/5) / (1/20)</gadget> <output>4</output> <gadget id="calculator">150 * 4</gadget> <output>600</output> <gadget id="calculator">600 - 150</gadget> <output>450</output> <result>450</result>',
        "chosen": 'The volume of the pesticide solution is 150 * 0.2 =<gadget id="calculator">150*0.2</gadget><output>30</output> 30 kg. The volume of the diluted drug solution is 30 / 0.05 =<gadget id="calculator">30/0.05</gadget><output>600</output> 600 kg. Thus, you need to add 600 - 150 =<gadget id="calculator">600-150</gadget><output>450</output> 450 kg of water.<result>450</result>',
    },
    {
        "prompt": "A supermarket buys 5 tons of rice and sells 2,000 kilograms in 5 days. How many kilograms are left?",
        "rejected": '<gadget id="calculator">5 * 1_000</gadget> <output>5_000</output> <gadget id="calculator">5_000 - 2_000</gadget> <output>3_000</output> <result>3_000</result>',
        "chosen": 'Five tons of rice is 5 x 1_000 =<gadget id="calculator">5*1_000</gadget><output>5_000</output> 5000 kilograms. After selling 2,000 kilograms, there will be 5000 - 2000 =<gadget id="calculator">5000-2000</gadget><output>3_000</output> 3000 kilograms left.<result>3_000</result>',
    },
    {
        "prompt": "The vegetable market shipped back 1,200 kilograms of eggplants. The tomatoes shipped back were eggplant (1/3). How many kilograms are tomatoes?",
        "rejected": '<gadget id="calculator">1 / 3</gadget> <output>1/3 = around 0.333333</output> <gadget id="calculator">1_200 * (1/3)</gadget> <output>400</output> <result>400</result>',
        "chosen": 'If eggplant weighs 1,200 kilograms, then tomatoes weigh 1 / 3 * 1,200 =<gadget id="calculator">1/3*1200</gadget><output>400</output> 400 kilograms.<result>400</result>',
    },
    {
        "prompt": "An electrical appliance factory originally planned to produce 4,200 refrigerators in 21 days. The actual daily production of 210 units, the actual completion of the task a few days earlier than the original plan?",
        "rejected": '<gadget id="calculator">4_200 / 210</gadget> <output>20</output> <gadget id="calculator">21 - 20</gadget> <output>1</output> <result>1</result>',
        "chosen": 'The total number of units to be produced is 4200 / 210 =<gadget id="calculator">4200/210</gadget><output>20</output> 20 units/day. Thus, the completion of the task can be completed in 21 - 20 =<gadget id="calculator">21-20</gadget><output>1</output> 1 day early.<result>1</result>',
    },
]

valid_dataset = [
    {
        "prompt": "In order to help the victims of the earthquake in Sichuan, the factory rushed to make a batch of disaster relief tents. The first workshop completed (1/5) of this batch of tents, the second workshop completed (1/4) of this batch of tents, and the remaining batch of tents What percentage of it is not completed?",
        "chosen": '<gadget id="calculator">1 / 5</gadget> <output>1/5 = around 0.2</output> <gadget id="calculator">1 / 4</gadget> <output>1/4 = around 0.25</output> <gadget id="calculator">1 - (1/5) - (1/4)</gadget> <output>11/20 = around 0.55</output> <result>11/20 = around 0.55</result>',
        "rejected": "",
    },
    {
        "prompt": "A 0.15-hectare paddy field, with a row spacing of 2 decimeters and a hole spacing of 15 cm, how many rice seedlings can be transplanted in total?",
        "chosen": '<gadget id="calculator">15 * 10_000</gadget> <output>150_000</output> <gadget id="calculator">0.15 * 10</gadget> <output>1.5</output> <gadget id="calculator">2 * 1.5</gadget> <output>3</output> <gadget id="calculator">150_000 / 3</gadget> <output>50_000</output> <result>50_000</result>',
        "rejected": "",
    },
    {
        "prompt": "Divide 30kg of sugar into how many bags of equal mass, the mass of each bag is (10/3) kg.",
        "chosen": '<gadget id="calculator">10 / 3</gadget> <output>10/3 = around 3.333333</output> <gadget id="calculator">30 / (10/3)</gadget> <output>9</output> <result>9</result>',
        "rejected": "",
    },
    {
        "prompt": "Xiao Ming bought 7 CDs and 42 floppy disks for 252 yuan, 4 yuan for each floppy disk, how much is a CD.",
        "chosen": '<gadget id="calculator">42 * 4</gadget> <output>168</output> <gadget id="calculator">252 - 168</gadget> <output>84</output> <gadget id="calculator">84 / 7</gadget> <output>12</output> <result>12</result>',
        "rejected": "",
    }
]


dummy_dataset = datasets.Dataset.from_list(dummy_dataset)
valid_dataset = datasets.Dataset.from_list(valid_dataset)

dummy_dataset, valid_dataset

(Dataset({
     features: ['prompt', 'rejected', 'chosen'],
     num_rows: 4
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 4
 }))

In [7]:
training_args = transformers.Seq2SeqTrainingArguments(
    max_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=5,
    output_dir="./output",
    report_to="none",
    remove_unused_columns=True,
    predict_with_generate=True,
    include_inputs_for_metrics=True,
)

In [8]:
compute_metrics = gadgets.metrics.MonitorMetrics(
    tokenizer,
    source_ds_col=["ape210k"] * len(valid_dataset),
    eval_ds_inputs=None,
    log_predictions=True
)

In [9]:
trainer = gadgets.dpo_trainer.DPOTrainer(
    model,
    args=training_args,
    train_dataset=dummy_dataset,
    eval_dataset=valid_dataset,
    max_length=756,
    max_prompt_length=756,
    max_target_length=756,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/home/xkadlci2/calc-x/gadgets/dpo_trainer.py:325: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your Seq2SeqTrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [10]:
compute_metrics.set_eval_ds_inputs(trainer.eval_dataset["prompt_input_ids"])

In [11]:
trainer.train()

Step,Training Loss,Validation Loss,Ape210k Rouge1,Ape210k Rouge2,Ape210k Rougel,Ape210k Rougelsum,Ape210k Sacrebleu,Ape210k Num Tokens,Ape210k Num Gadget Calls Pred,Ape210k Num Gadget Calls True,Ape210k Correct Results,Avg Correct Results
5,No log,0.042748,0.913866,0.878205,0.892857,0.892857,79.831281,143.000000,3.500000,3.000000,0.750000,0.750000
10,No log,0.042398,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
15,No log,0.042408,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
20,No log,0.042515,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
25,No log,0.042635,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
30,No log,0.042751,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
35,No log,0.042854,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
40,No log,0.042940,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
45,No log,0.043006,0.951031,0.892105,0.925258,0.925258,90.594101,125.000000,3.000000,3.000000,0.750000,0.750000
50,No log,0.043060,0.927273,0.888889,0.918182,0.918182,84.464190,134.250000,3.250000,3.000000,0.750000,0.750000


TrainOutput(global_step=100, training_loss=0.007769244313240051, metrics={'train_runtime': 1375.5751, 'train_samples_per_second': 0.291, 'train_steps_per_second': 0.073, 'total_flos': 75622411468800.0, 'train_loss': 0.007769244313240051, 'epoch': 100.0})